# blender_utils

参考: [zju3dv/pvnet-rendering](https://github.com/zju3dv/pvnet-rendering/blob/master/base_utils.py)

In [1]:
import os
import sys
sys.path.append('..')
sys.path.append('../../')

import numpy as np

from src.config.config import cfg
from src.datasets.LineMod.LineModDB import LineModDB
from src.utils.base_utils import read_ply_model

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

## Setup

In [3]:
pvnet_linemod_dir = cfg.PVNET_LINEMOD_DIR
linemod_dir = cfg.LINEMOD_DIR
obj_name = 'ape'

## 1. ModelAligner Test

In [4]:
from blender_utils import ModelAligner

### インスタンス生成

In [5]:
aligner = ModelAligner(obj_name=obj_name,
                                                pvnet_linemod_dir=pvnet_linemod_dir,
                                                linemod_dir=linemod_dir)

### 1.1. get_translation_transform Test

In [6]:
translation_transform = aligner.get_translation_transform()

print('translation_transform type')
print(type(translation_transform))
print('translation_transform =')
print(translation_transform)

translation_transform type
<class 'numpy.ndarray'>
translation_transform =
[ 0.00642697  0.00273947 -0.0337945 ]


## 2. PoseTransformer

In [7]:
from blender_utils import PoseTransformer

### 2.1. インスタンス生成

In [8]:
pose_transformer = PoseTransformer(obj_name,
                                                                               pvnet_linemod_dir,
                                                                               linemod_dir)

### 2.2. orig_pose_to_blender_pose Test

In [9]:
from src.datasets.LineMod.LineModDB import read_pose

In [10]:
img_num = 0
rt_dir = os.path.join(linemod_dir,
                                          obj_name,
                                          'data')
pose = read_pose(os.path.join(rt_dir, 'rot{}.rot'.format(img_num)),
                                      os.path.join(rt_dir, 'tra{}.tra'.format(img_num)))
print('pose =', pose)

RT = pose_transformer.orig_pose_to_blender_pose(pose).astype(np.float32)
print('RT =', RT)
#print(RT)

pose = [[-0.0963063  0.994044  -0.0510079 -0.111814 ]
 [-0.573321  -0.0135081  0.81922   -0.0783622]
 [ 0.813651   0.10814    0.571207   1.03612  ]]
RT = [[-0.0963063  -0.994044    0.0510079  -0.10798602]
 [-0.573321    0.0135081  -0.81922    -0.10976905]
 [ 0.813651   -0.10814    -0.571207    1.0223418 ]]


## 2.3. orig_pose_to_blender_euler Test

In [12]:
img_num = 0
rt_dir = os.path.join(linemod_dir,
                                          obj_name,
                                          'data')
pose = read_pose(os.path.join(rt_dir, 'rot{}.rot'.format(img_num)),
                                      os.path.join(rt_dir, 'tra{}.tra'.format(img_num)))
print('pose =', pose)

euler = pose_transformer.orig_pose_to_blender_euler(pose)
print('euler =', euler)

pose = [[-0.0963063  0.994044  -0.0510079 -0.111814 ]
 [-0.573321  -0.0135081  0.81922   -0.0783622]
 [ 0.813651   0.10814    0.571207   1.03612  ]]
euler = [172.42935241  34.83444354   3.56286401]
